In [7]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import load_model


In [8]:
# Define the input size expected by Xception
input_shape = (71, 71, 3)  # Xception requires input images to be at least 71x71

# Load the pre-trained Xception model without the top classification layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom classification layers for your flower classes
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # 5 output classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with learning_rate instead of lr
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


In [9]:

training_set = train_datagen.flow_from_directory(
    r"C:\Users\Aditi\Desktop\Model2\train",
    target_size=(71, 71),
    batch_size=28,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    r"C:\Users\Aditi\Desktop\Model2\test",
    target_size=(71, 71),
    batch_size=28,
    class_mode='categorical'
)



Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [10]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set),
    callbacks=[early_stopping]
)

Epoch 1/10
143/143 [==============================] - 166s 1s/step - loss: 1.0273 - accuracy: 0.6087 - val_loss: 0.5645 - val_accuracy: 0.7980
Epoch 2/10
143/143 [==============================] - 158s 1s/step - loss: 0.3972 - accuracy: 0.8620 - val_loss: 0.4686 - val_accuracy: 0.8630
Epoch 3/10
143/143 [==============================] - 152s 1s/step - loss: 0.2483 - accuracy: 0.9158 - val_loss: 0.3090 - val_accuracy: 0.9010
Epoch 4/10
143/143 [==============================] - 156s 1s/step - loss: 0.1681 - accuracy: 0.9427 - val_loss: 0.3631 - val_accuracy: 0.8920
Epoch 5/10
143/143 [==============================] - 160s 1s/step - loss: 0.1316 - accuracy: 0.9548 - val_loss: 0.2561 - val_accuracy: 0.9220
Epoch 6/10
143/143 [==============================] - 158s 1s/step - loss: 0.0970 - accuracy: 0.9700 - val_loss: 0.2748 - val_accuracy: 0.9350
Epoch 7/10
143/143 [==============================] - 161s 1s/step - loss: 0.0896 - accuracy: 0.9705 - val_loss: 0.2403 - val_accuracy: 0.9380

In [11]:

# Evaluate the model and print the accuracy
test_loss, test_acc = model.evaluate(test_set)
print(f"Testing accuracy: {test_acc * 100:.2f}%")


36/36 [==============================] - 9s 240ms/step - loss: 0.2563 - accuracy: 0.9370
Testing accuracy: 93.70%


In [12]:
model.save('Model2.h5')
loaded_model = load_model('Model2.h5')
loaded_model.summary()


C:\Users\Aditi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 71, 71, 3)]          0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 35, 35, 32)           864       ['input_3[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 35, 35, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 35, 35, 32)           0         ['block1_conv1_bn[0][0]'